# The MPS Calculator — Overview

**Goal.**  
Convert an $n$-qubit statevector $\psi \in \mathbb{C}^{2^n}$ into a **Matrix Product State (MPS)** using sequential SVD; inspect the singular values at the middle cut (entanglement profile); and use the tensor networking for reconstruction checks and entropy calculations.

**What this notebook takes in / produces**
- **Input:** a complex text file containing amplitudes of a state $|\psi\rangle$ (from your Clifford/FGS/Haar generators).  
- **Output:** a vector `MPS` of site tensors $A^{[k]} \in \mathbb{C}^{D_{k-1}\times 2 \times D_k}$, a plot of singular values at the middle bond, and helper metrics (Rényi entropies, reconstruction error).

**Why MPS?**  
MPS exposes the **Schmidt spectrum** across each bipartition via singular values. The **bond dimension** $\chi=\max_k D_k$ encodes how entangled the state is.



## MPS construction (Julia function)

**Idea .**  
Sweep left→right. At step $k$:
1. Reshape the current vector into a matrix $(D_{k-1}\!\cdot\!2) \times D_{\text{right}}$ that isolates site $k$.
2. Compute SVD $$\Psi_k = U_k \, \mathrm{diag}(S_k) \, V_k^\dagger.$$
3. Keep the top $r$ singular values with $|S_k|>\texttt{cutoff}$ (noise control / entanglement truncation).
4. Store $A^{[k]}=\mathrm{reshape}(U_k^{(:,1:r)},\, D_{k-1}, 2, r)$ and pass $\Psi_{k+1}=\mathrm{diag}(S_k^{(1:r)}) \, (V_k^{(:,1:r)})^\dagger$ to the next step.
5. At the end, reshape the remainder into $A^{[n]} \in \mathbb{C}^{D_{n-1}\times 2 \times 1}$.

**What the code does**
- Computes $n=\log_2(\texttt{length}(\psi))$ and asserts power-of-two length.
- Loops over sites $k=1,\dots,n-1$, reshaping, SVD’ing, **counting rank** $r$ above `cutoff`, truncating $(U,S,V)$, and pushing $A^{[k]}$ into `mps`.
- At $k=\lceil n/2\rceil$ it **records/plots** the middle singular values $S_{k^\star}$ (where entanglement typically peaks).
- Finishes with the last tensor and returns the `mps` vector.

**Reading the printed shapes.**  
Lines like `A[5] shape: (16, 2, 32)` mean left bond $D_{4}=16$, physical dimension $2$, right bond $D_{5}=32$. The sequence $\{D_k\}$ is your running estimate of entanglement across cuts; $\max_k D_k$ is $\chi$.


In [1]:
using LinearAlgebra
using Plots: plot, savefig
dir = "/Users/arya/Documents/Python/DATA/"


# Convert a state vector into its Matrix Product State (MPS) representation
function mps_function(psi::Vector{ComplexF64}, cutoff::Float64)
    n = round(Int, log2(length(psi)))  # Number of qubits
    @assert 2^n == length(psi) "Length of psi must be a power of 2"

    mps = []
    D_prev = 1 # Initial bond dimension

    for k in 1:n-1
        d_left = D_prev
        d_phys = 2 # Physical dimension (2 for qubits)
        d_right = length(psi) ÷ (d_left * d_phys)

        psi_matrix = reshape(psi, (d_left * d_phys, d_right))
        U, S, V = svd(psi_matrix)
        
       
      
        if k == ceil(Int, n/2)
            global middle_singular_values = copy(S)
            plt = plot(
                S,
                marker = :circle,
                xlabel = "Number of Singular Values",
                ylabel = "Singular Values",
                title = "Singular Values at Step $k (Middle Site)",
                yscale = :log10,
                ylims = (1e-25, 1e5),
                #yformatter = y -> @sprintf("%.5e", y),   # <--- show 2 decimal digits in scientific notation
                legend = false
            )
            savefig(plt, dir * "singular_values_plot(FGS).pdf")
        end

        

        # Find how many singular values are above cutoff
        r = count(s -> abs(s) > cutoff, S)
        if r == 0
            error("All singular values below cutoff at step $k — increase cutoff or check state")
        end

        # Truncate U, S, V to rank r  (non zero singular values)
        U_trunc = U[:, 1:r]
        S_trunc = S[1:r]
        V_trunc = V[:, 1:r]

        A = reshape(U_trunc, d_left, d_phys, r)
        push!(mps, A)

        psi = Diagonal(S_trunc) * V_trunc'
        D_prev = r
    end

    # Final tensor
    A_last = reshape(psi, D_prev, 2, 1)
    push!(mps, A_last)

    return mps
end



# READING COMPLEX TEXT FILE (2 Functions) 
function load_complex_vector_from_txt(filename::String)
    psi = ComplexF64[]
    open(filename, "r") do file
        for line in eachline(file)
            s = replace(line, "im" => "")  # remove "im"
            parts = split(s, "+")
            if length(parts) == 2
                real_part = parse(Float64, strip(parts[1]))
                imag_part = parse(Float64, strip(parts[2]))
                push!(psi, ComplexF64(real_part, imag_part))
            end
        end
    end
    return psi
end

function load_complex_vector_from_txt_2(filename::String)
    psi = ComplexF64[]
    open(filename, "r") do file
        for line in eachline(file)
            # Remove "im" and index number
            parts = split(strip(line), r"\s+")
            if length(parts) >= 4
                real_str = parts[1+1]  # after the index
                sign = parts[2+1]      # '+' or '-'
                imag_str = parts[3+1]

                real_part = parse(Float64, real_str)
                imag_part = parse(Float64, imag_str)
                imag_part *= (sign == "+" ? 1 : -1)

                push!(psi, ComplexF64(real_part, imag_part))
            end
        end
    end
    return psi
end


load_complex_vector_from_txt_2 (generic function with 1 method)

In [2]:
# APLLYING THE MPS CALCULATER ON DIFFERENT STATEVECTORS
psi = load_complex_vector_from_txt(dir * "cl.txt")
MPS = mps_function(psi, 1e-8)
for (i, A) in enumerate(MPS)
    println("A[$i] shape: ", size(A))
end

A[1] shape: (1, 2, 2)
A[2] shape: (2, 2, 4)
A[3] shape: (4, 2, 8)
A[4] shape: (8, 2, 16)
A[5] shape: (16, 2, 32)
A[6] shape: (32, 2, 16)
A[7] shape: (16, 2, 8)
A[8] shape: (8, 2, 4)
A[9] shape: (4, 2, 2)
A[10] shape: (2, 2, 1)


## Rényi entropies from singular values

**Definition**  
For singular values $\{\lambda_i\}$ at a chosen cut,
$$
S_\alpha \;=\; \frac{1}{1-\alpha}\,\log\!\left(\sum_i \lambda_i^{\,2\alpha}\right),
\qquad \alpha\ge 1.
$$
The code:
- Squares $\lambda$ to probabilities $p_i=\lambda_i^2$, handles $0\log 0$ safely, returns $S_1$ (von Neumann) and $S_\alpha$ for $\alpha=2{:}10$.
- Prints the middle-cut singular values and the vector of entropies.

**What to expect.**  
- **Stabilizer states:** flat spectrum on $r$ values $\Rightarrow S_\alpha=\log r$ for all $\alpha$.  
- **Haar-typical:** strict hierarchy $S_\infty<\cdots<S_2<S_1\lesssim \log d_A$ (Page-like behavior).  
- **FGS:** $S_\alpha$ grows toward **half-filling** and then decreases symmetrically.


In [3]:
# Calculate Renyi Entropies from Singular Values
function renyi_entropies(S::AbstractVector)
    p = S.^2
    p ./= sum(p)
    Ren_En = zeros(Float64, 10)
    #Van Neumann Entropy
    Ren_En[1] = -sum(pi == 0 ? 0.0 : pi * log(pi) for pi in p)          #Note to myself: Ternary Operator
    #Renyi Entropies
    for n in 2:10
        Ren_En[n] = (1 / (1 - n)) * log(sum(p .^ n))
    end

    return Ren_En
end



# Checking the singular values
println("Singular Values at the Middle Site:", middle_singular_values)

# Renyi Entropy of the state: 
E = renyi_entropies(middle_singular_values)

Singular Values at the Middle Site:[0.1767760000000001, 0.1767760000000001, 0.1767760000000001, 0.17677600000000007, 0.17677600000000007, 0.17677600000000007, 0.17677600000000007, 0.17677600000000007, 0.17677600000000007, 0.17677600000000007, 0.17677600000000007, 0.17677600000000002, 0.17677600000000002, 0.17677600000000002, 0.17677600000000002, 0.17677600000000002, 0.17677600000000002, 0.17677600000000002, 0.176776, 0.176776, 0.176776, 0.176776, 0.176776, 0.176776, 0.176776, 0.176776, 0.176776, 0.17677599999999996, 0.17677599999999996, 0.17677599999999996, 0.17677599999999993, 0.1767759999999999]


10-element Vector{Float64}:
 3.465735902799726
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257
 3.465735902799726
 3.465735902799726
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257
 3.4657359027997257

## SERs — Stabilizer Rényi Entropy and Magic

**Goal.**  
Quantify how far a state $|\psi\rangle$ is from the stabilizer manifold.

**Stabilizer Rényi Entropy (SRE):**
$$
m_n(|\psi\rangle) = d^{-n} \sum_{\sigma} |\langle\psi | \sigma | \psi \rangle|^{2n}, \qquad
\mathcal{M}_n(|\psi\rangle) = \frac{1}{1-n}\log m_n(|\psi\rangle) - \log d.
$$
Here $\sigma$ runs over all **Pauli strings** forming the stabilizer basis of dimension $d=2^N$.  
In practice, each $\sigma$ must be represented as an **MPO** acting on the MPS to evaluate the overlaps $\langle \psi|\sigma\rangle$.

- For exact stabilizer states, only one term contributes $\Rightarrow \mathcal{M}_n = 0$.  
- For non-Clifford (magical) states, multiple overlaps are nonzero $\Rightarrow \mathcal{M}_n > 0$.

$\mathcal{M}_n$ grows as $T$-gates or other non-Clifford resources are introduced.


### Building the Pauli Strings

In [4]:
using LinearAlgebra

#Importing the stabilizer group of the state
labels = readlines("/Users/arya/Documents/Python/DATA/P_labels.txt")

#defining Pauli Matrices
s_x = ComplexF64[0 1; 1 0]
s_y = ComplexF64[0 -im; im 0]
s_z = ComplexF64[1 0; 0 -1]
s_i = Matrix{ComplexF64}(I, 2, 2)

# Map labels to matrices
if !@isdefined pauli_map
    const pauli_map = Dict{Char, Matrix{ComplexF64}}('X'=>s_x, 'Y'=>s_y, 'Z'=>s_z, 'I'=>s_i)  
end 

# Convert Pauli string to MPO
function pauli_to_mpo(label::String)
    s = strip(label)
    coef = startswith(s, '-') ? (s = s[2:end]; -1.0) : 1.0
    mpo = [reshape(pauli_map[c], 1,1,2,2) for c in s]  # convert Char→String
    mpo[1] .*= coef
    return mpo
end

mpo = pauli_to_mpo(labels[1])

10-element Vector{Array{ComplexF64, 4}}:
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]
 [1.0 + 0.0im;;; 0.0 + 0.0im;;;; 0.0 + 0.0im;;; 1.0 + 0.0im]

In [ ]:
Pauil_MPOs = []
for label in labels
    mpo = pauli_to_mpo(label)
    push!(Pauil_MPOs, mpo)
end

## Sanity check — Reconstructing the state and error budget

**Goal.**  
Confirm the MPS truly represents $|\psi\rangle$ (within the chosen cutoff).

**What the code does**
- `contract_mps(MPS)`: sequentially contracts site tensors with `@tensor` to rebuild the full statevector.  
- `cutoff_complex_parts(A, ε)`: zeroes tiny real/imag parts to avoid printing noise.  
- Compares the reconstructed vector to the original via `norm(re_psi - psi)`.



In [ ]:
# Reconstructing the statevector by contracting MPS elements
using TensorOperations


# Contract mps
function contract_mps(MPS::Vector)
    psi = MPS[1]  # Start with the first site tensor
0
    for i in 2:length(MPS)
        T = MPS[i]

        
        @tensor psi_new[a, p1, p2, c] := psi[a, p1, b] * T[b, p2, c]

        
        d1 = size(psi, 2)
        d2 = size(T, 2)
        right_bond = size(psi_new, 4)

        psi = reshape(psi_new, 1, d1 * d2, right_bond)
    end

    
    return reshape(psi, :)
end

# Cutting off the complex numbers
function cutoff_complex_parts(A::Array, cutoff::Float64 = 1e-12)
    re = real.(A)
    im = imag.(A)

    re[abs.(re) .< cutoff] .= 0.0
    im[abs.(im) .< cutoff] .= 0.0

    A .= complex.(re, im)
end


# Applying the reconstruction
re_psi_1 = contract_mps(MPS)
re_psi = cutoff_complex_parts(re_psi_1, 1e-8)

# Checking the difference

norm(re_psi - psi)

contract_mps (generic function with 1 method)